In [1]:
import pandas as pd
import os
import numpy as np
import copy
import random

In [2]:
def make_derangement(num_index, how_many):
    idx = np.arange(num_index)
    
    sf = copy.deepcopy(idx)
    results = []
    for _ in range(min(num_index-1, how_many)):
        # make a derangement
        while True:
            np.random.shuffle(sf)
            if not np.any((sf == idx)):
                results.append(sf.tolist())
                break
    return results

print(make_derangement(10, 5))
print()
print(make_derangement(3, 5))

[[4, 8, 6, 7, 9, 1, 3, 0, 2, 5], [9, 0, 7, 1, 2, 8, 5, 4, 3, 6], [6, 7, 3, 0, 1, 4, 8, 5, 9, 2], [3, 0, 1, 4, 9, 6, 8, 5, 2, 7], [1, 3, 7, 8, 5, 2, 9, 0, 4, 6]]

[[2, 0, 1], [2, 0, 1]]


In [3]:
no_new_whale_list = pd.read_csv('../../data/all/train_no_new_whale.csv')
new_whale_list = pd.read_csv('../../data/all/new_whale.csv')
print(no_new_whale_list.head())
print(new_whale_list.head())

           Image         Id
0  0000e88ab.jpg  w_f48451c
1  0001f9222.jpg  w_c3d896a
2  00029d126.jpg  w_20df2c5
3  000a6daec.jpg  w_dd88965
4  0016b897a.jpg  w_64404ac
           Image         Id
0  00050a15a.jpg  new_whale
1  0005c1ef8.jpg  new_whale
2  0006e997e.jpg  new_whale
3  000f0f2bf.jpg  new_whale
4  001d7450c.jpg  new_whale


In [4]:
no_new_whale_list_len = no_new_whale_list.shape[0]
print(no_new_whale_list_len)

15697


In [5]:
# get all ids for known whales
known_ids = no_new_whale_list.Id.unique().tolist()
print(known_ids[:10])
print(len(known_ids))

['w_f48451c', 'w_c3d896a', 'w_20df2c5', 'w_dd88965', 'w_64404ac', 'w_a6f9d33', 'w_581ba42', 'w_cb622a2', 'w_d3b46e7', 'w_8cad422']
5004


## make match pairs

In [6]:
# make match pairs, in form: im_a, im_b, id_a, id_b, label, label == 1 means they're
# from a same whale

match_pairs = pd.DataFrame(columns=['im_a', 'im_b', 'id_a', 'id_b', 'label'])

for id in known_ids:
    one_whale = no_new_whale_list[no_new_whale_list.Id == id]
    one_whale = one_whale.reset_index(drop=True)
    one_whale_len = one_whale.shape[0]
    
    if  one_whale_len == 1:
        continue # drop just ID which just has only one image
        #im_a = one_whale.Image[0]
        #im_b = one_whale.Image[0]
        #id_a = id
        #id_b = id
        #label = int(1)
        #match_pairs = match_pairs.append({'im_a':im_a, 'im_b':im_b, 'id_a':id_a,
        #                                 'id_b':id_b, 'label':label},
        #                                ignore_index=True)
    else:
        derangements = make_derangement(one_whale_len, 10)
        for drg in derangements:
            one_drged = one_whale.iloc[drg, :]
            match_pairs = match_pairs.append(
                pd.DataFrame({'im_a':one_whale.Image.iloc[:].as_matrix().tolist(),
                 'im_b':one_drged.Image.iloc[:].as_matrix().tolist(), 
                 'id_a':id, 'id_b':id, 'label':int(1)}),
                ignore_index=True)

print(match_pairs.shape)

(76786, 5)


In [7]:
match_pairs.to_csv('../../data/all/match_pairs_10.csv', index=False)

## make different pairs

sicne we made 76786 match_paris, we now will make 5 times num of diffrent paris, for 15697 images in no_new_whale_list, we make 25 pairs for each image, 13 from no_new_whale_list, another 12 from new_whale_list

In [8]:
different_pairs = pd.DataFrame(columns=['id_a', 'id_b', 'im_a', 'im_b', 'label'])

for i in range(no_new_whale_list_len):
    one_whale = no_new_whale_list.iloc[i,:]
    id_a = one_whale.Id
    im_a = one_whale.Image
    known_candidate = no_new_whale_list[no_new_whale_list.Id != id_a]
    idxs_known = np.random.choice(np.arange(known_candidate.shape[0]).tolist(), 13)
    idxs_new_whale = np.random.choice(np.arange(new_whale_list.shape[0]).tolist(), 12)
    concated = pd.concat([known_candidate.iloc[idxs_known, :],
                          new_whale_list.iloc[idxs_new_whale, :]],
                        ignore_index=True)
    different_pairs = different_pairs.append(
        pd.DataFrame({'id_a': id_a,
                      'id_b': concated.Id.iloc[:].as_matrix().tolist(),
                      'im_a': im_a,
                      'im_b': concated.Image.iloc[:].as_matrix().tolist(),
                      'label': int(0)}), 
        ignore_index=True)

print(different_pairs.shape)

(392425, 5)


In [9]:
print(different_pairs.iloc[200:400])
print(different_pairs.iloc[-400:-200])
different_pairs.to_csv('../../data/all/different_pairs_10.csv', index=False)

          id_a       id_b           im_a           im_b label
200  w_d3b46e7  w_1ca9ab1  00357e37a.jpg  bb86f0613.jpg     0
201  w_d3b46e7  w_2f350be  00357e37a.jpg  48dcbcac0.jpg     0
202  w_d3b46e7  w_ef83760  00357e37a.jpg  f924801c6.jpg     0
203  w_d3b46e7  w_1b0db58  00357e37a.jpg  f36ec15ff.jpg     0
204  w_d3b46e7  w_5793f1c  00357e37a.jpg  d0b96c2e9.jpg     0
205  w_d3b46e7  w_456646a  00357e37a.jpg  4a9ad3167.jpg     0
206  w_d3b46e7  w_4441671  00357e37a.jpg  be49fc1a2.jpg     0
207  w_d3b46e7  w_1ca7433  00357e37a.jpg  6cf8a08d1.jpg     0
208  w_d3b46e7  w_03e3e5e  00357e37a.jpg  266992f6a.jpg     0
209  w_d3b46e7  w_90abfeb  00357e37a.jpg  2f510dcb4.jpg     0
210  w_d3b46e7  w_69b5ace  00357e37a.jpg  26d46f62c.jpg     0
211  w_d3b46e7  w_e8b82f6  00357e37a.jpg  1ff9a9a3b.jpg     0
212  w_d3b46e7  w_0fdf741  00357e37a.jpg  5003d7331.jpg     0
213  w_d3b46e7  new_whale  00357e37a.jpg  523653a6b.jpg     0
214  w_d3b46e7  new_whale  00357e37a.jpg  3257f472c.jpg     0
215  w_d

In [10]:
merged_list = pd.concat([match_pairs, different_pairs], ignore_index=True)
shuffle_idx = np.arange(merged_list.shape[0])
np.random.shuffle(shuffle_idx)
np.random.shuffle(shuffle_idx)
np.random.shuffle(shuffle_idx)
merged_list = merged_list.iloc[shuffle_idx,:]
merged_list = merged_list.reset_index(drop=True)
print(merged_list.shape)
merged_list.iloc[:20,:]

(469211, 5)


,id_a,id_b,im_a,im_b,label
0,w_480d478,new_whale,e36654163.jpg,7e453ebba.jpg,0
1,w_e53ba4a,new_whale,b7e517743.jpg,b8e4d6d78.jpg,0
2,w_f765256,new_whale,212b9bd39.jpg,2cc7e409a.jpg,0
3,w_01f14e1,w_f829227,c1e1d7f3e.jpg,d844da11a.jpg,0
4,w_803ad0d,w_179c9f0,4eb9b51de.jpg,f69c572db.jpg,0
5,w_70d0b3c,new_whale,2d07ab7d2.jpg,fe0241a8f.jpg,0
6,w_cbb15d2,new_whale,ec0d8583d.jpg,22cb0c36f.jpg,0
7,w_4e1e04d,new_whale,00b92e9bf.jpg,1e55ac126.jpg,0
8,w_c25d2d1,new_whale,19d6253de.jpg,946fc7503.jpg,0
9,w_e2372d6,w_5a9abcf,241aeee0f.jpg,ac29bbe2e.jpg,0


In [11]:
merged_list.to_csv('../../data/all/siamese_pairs_10.csv', index=False)

### split siamese_pairs_10_all into train and val split

In [13]:
siamese_paris_all = pd.read_csv('../../data/all/siamese_pairs_10.csv')
print(siamese_paris_all.shape)
siamese_paris_all.head()

(469211, 5)


,id_a,id_b,im_a,im_b,label
0,w_480d478,new_whale,e36654163.jpg,7e453ebba.jpg,0
1,w_e53ba4a,new_whale,b7e517743.jpg,b8e4d6d78.jpg,0
2,w_f765256,new_whale,212b9bd39.jpg,2cc7e409a.jpg,0
3,w_01f14e1,w_f829227,c1e1d7f3e.jpg,d844da11a.jpg,0
4,w_803ad0d,w_179c9f0,4eb9b51de.jpg,f69c572db.jpg,0


In [14]:
all_idx = np.arange(siamese_paris_all.shape[0])
print(all_idx[:20])
np.random.shuffle(all_idx)
np.random.shuffle(all_idx)
np.random.shuffle(all_idx)
print(all_idx[:20])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[401943  73790 189878 283706 202716 270863 293660  15707 201803  79844
 284063  63645 447586 369383  45009 396151 185722 268636 182956  95472]


In [15]:
train_split_long = int(siamese_paris_all.shape[0] * 0.9)
train_split_idx = all_idx[:train_split_long]
val_split_idx = all_idx[train_split_long:]
print(train_split_idx.shape, train_split_idx, train_split_idx.min(), train_split_idx.max())
print(val_split_idx.shape, val_split_idx, val_split_idx.min(), val_split_idx.max())

(422289,) [401943  73790 189878 ... 415880 454870 316504] 0 469210
(46922,) [104220  49251 224388 ... 247270  87957 220755] 12 469208


In [16]:
train_split = siamese_paris_all.iloc[train_split_idx]
val_split = siamese_paris_all.iloc[val_split_idx]

In [17]:
print(train_split.shape)
train_split.head()

(422289, 5)


,id_a,id_b,im_a,im_b,label
401943,w_ab44ae4,new_whale,be51fb093.jpg,e106eb7b4.jpg,0
73790,w_20950a9,w_20950a9,d0e44ece4.jpg,a670c5bc2.jpg,1
189878,w_f91389a,w_ffa542b,f86bcd018.jpg,8abea550e.jpg,0
283706,w_ed1df50,new_whale,029449f85.jpg,d49a452aa.jpg,0
202716,w_d10cf66,w_1f1cee1,6f3df50a3.jpg,dd468c731.jpg,0


In [18]:
print(val_split.shape)
val_split.head()

(46922, 5)


,id_a,id_b,im_a,im_b,label
104220,w_c25d2d1,new_whale,3f22b5160.jpg,e88546ec8.jpg,0
49251,w_2b4b82e,new_whale,bcd1a1957.jpg,1b3d1fa16.jpg,0
224388,w_c6b4d61,w_c6b4d61,85369a41f.jpg,ed135b296.jpg,1
184958,w_695c604,w_b71ac68,8f8328950.jpg,25ff06acf.jpg,0
139412,w_fd51859,w_b3ca4b7,bc9a158e6.jpg,5117e04c0.jpg,0


In [19]:
train_split.to_csv('../../data/all/siamese_pairs_10_train.csv', index=False)
val_split.to_csv('../../data/all/siamese_pairs_10_val.csv', index=False)